# Model selection for knowledge graph embeddings

For hyperparameter optimisation random search is more optimal than grid search as the search space grows: *James Bergstra and Yoshua Bengio. Random search for hyper-parameter optimization. Journal of Machine Learning Research, 13(Feb):281–305, 2012.*

While this approach is not optimal, it is a strong baseline agains other more advanced methods such as Baysian optimisation: *Lisha Li and Kevin Jamieson. Hyperband: a novel bandit-based approach to hyperparameter optimization. Journal of Machine Learning Research, 18:1–52, 2018.*

In [1]:
import tensorflow as tf
import numpy as np
import pickle
np.random.seed(0)

from ampligraph.evaluation import train_test_split_no_unseen 
from ampligraph.latent_features import RandomBaseline, TransE, DistMult, ComplEx, HolE, ConvE, ConvKB
from ampligraph.latent_features import save_model
from ampligraph.evaluation import evaluate_performance, select_best_model_ranking, mr_score, mrr_score, hits_at_n_score
from ampligraph.latent_features import save_model, restore_model

## Data retrieval

In [2]:
#data = np.loadtxt("family_subset.txt", dtype = 'object')

In [3]:
data = np.loadtxt("family_subset.txt", dtype = 'object')
X_train = data[:100]
X_val = data[100:150]
X_test = data[150:190]

In [4]:
data.shape

(258235, 3)

## Metrics

In [5]:
"""
# borrowed from https://github.com/Accenture/AmpliGraph/blob/master/ampligraph/evaluation/protocol.py
def evaluation(ranks):
        mrr = mrr_score(ranks)
        mr = mr_score(ranks)
        hits_1 = hits_at_n_score(ranks, n=1)
        hits_3 = hits_at_n_score(ranks, n=3)
        hits_10 = hits_at_n_score(ranks, n=10)
        test_evaluation = {
            "mrr": mrr,
            "mr": mr,
            "hits_1": hits_1,
            "hits_3": hits_3,
            "hits_10": hits_10
        }
        return test_evaluation
    
def get_metrics(model, test_data, complete_data):
    ranks = evaluate_performance(complete_data, model=model,
                                         filter_triples=complete_data, verbose=False,
                                         entities_subset=None,
                                         use_default_protocol=False,
                                         corrupt_side='s,o')
    return evaluation(ranks)
"""

'\n# borrowed from https://github.com/Accenture/AmpliGraph/blob/master/ampligraph/evaluation/protocol.py\ndef evaluation(ranks):\n        mrr = mrr_score(ranks)\n        mr = mr_score(ranks)\n        hits_1 = hits_at_n_score(ranks, n=1)\n        hits_3 = hits_at_n_score(ranks, n=3)\n        hits_10 = hits_at_n_score(ranks, n=10)\n        test_evaluation = {\n            "mrr": mrr,\n            "mr": mr,\n            "hits_1": hits_1,\n            "hits_3": hits_3,\n            "hits_10": hits_10\n        }\n        return test_evaluation\n    \ndef get_metrics(model, test_data, complete_data):\n    ranks = evaluate_performance(complete_data, model=model,\n                                         filter_triples=complete_data, verbose=False,\n                                         entities_subset=None,\n                                         use_default_protocol=False,\n                                         corrupt_side=\'s,o\')\n    return evaluation(ranks)\n'

In [6]:
test_metrics = {}

## Random Baseline
Random baseline requires no hyperparameter search as it assigns a pseudo-random score to triples.

In [7]:
model_class = RandomBaseline
param_grid = {
    "seed": 0
}
best_model, _, _, _, randomBaseline_mrr_test, _ = select_best_model_ranking(model_class, X_train, X_val, X_test,
                          param_grid,
                          max_combinations=1,
                          use_filter=True,
                          verbose=False,
                          early_stopping=False)

WARNING - All triples will be processed in the same batch (batches_count=1). When processing large graphs it is recommended to batch the input knowledge graph instead.


In [8]:
test_metrics["RandomBaseline"] = randomBaseline_mrr_test
save_model(best_model, './trained_models/RandomBaseline.pkl')
del best_model

## TransE

In [9]:
model_class = TransE
param_grid = {
    "batches_count": [50],
    "seed": 0,
     "epochs": [10],
     "k": [100, 200],
     "eta": [5, 10, 15],
     "loss": ["pairwise", "nll"],
     "loss_params": {
         "margin": [2]
     },
     "embedding_model_params": {
     },
     "regularizer": ["LP", None],
     "regularizer_params": {
         "p": [1, 3],
         "lambda": [1e-4, 1e-5]
     },
     "optimizer": ["adagrad", "adam"],
     "optimizer_params": {
         "lr": lambda: np.random.uniform(0.0001, 0.01)
     },
     "verbose": False
}

In [10]:
best_model, _, _, _, transE_mrr_test, _ = select_best_model_ranking(model_class, X_train, X_val, X_test,
                          param_grid,
                          max_combinations=6,
                          use_filter=True,
                          verbose=False,
                          early_stopping=False)

In [12]:
save_model(best_model, './trained_models/TransE.pkl')
test_metrics["TransE"] = transE_mrr_test
del best_model

## Distmult

In [13]:
model_class = DistMult
param_grid = {
    "batches_count": [50],
    "seed": 0,
    "epochs": [10],
    "k": [100, 200],
    "eta": [5, 10, 15],
    "loss": ["pairwise", "nll"],
    "loss_params": {
        "margin": [2]
    },
    "embedding_model_params": {
        
    },
    "regularizer": ["LP", None],
    "regularizer_params": {
        "p": [1, 3],
        "lambda": [1e-4, 1e-5]
    },
    "optimizer": ["adagrad", "adam"],
    "optimizer_params": {
        "lr": lambda: np.random.uniform(0.0001, 0.01)
    },
    "verbose": True
}

In [14]:
best_model, _, _, _, distMult_mrr_test, _ = select_best_model_ranking(model_class, X_train, X_val, X_test,
                          param_grid,
                          max_combinations=6,
                          use_filter=True,
                          verbose=False,
                          early_stopping=False)

100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 28.97it/s]


In [15]:
save_model(best_model, './trained_models/DistMult.pkl')
test_metrics["DistMult"] = distMult_mrr_test
del best_model

## ComplEx

In [16]:
model_class = ComplEx
param_grid = {
    "batches_count": [50],
    "seed": 0,
    "epochs": [10],
    "k": [100, 200],
    "eta": [5, 10, 15],
    "loss": ["pairwise", "nll"],
    "loss_params": {
        "margin": [2]
    },
    "embedding_model_params": {
        
    },
    "regularizer": ["LP", None],
    "regularizer_params": {
        "p": [1, 3],
        "lambda": [1e-4, 1e-5]
    },
    "optimizer": ["adagrad", "adam"],
    "optimizer_params": {
        "lr": lambda: np.random.uniform(0.0001, 0.01)
    },
    "verbose": True
}

In [17]:
best_model, _, _, _, complEx_mrr_test, _ = select_best_model_ranking(model_class, X_train, X_val, X_test,
                          param_grid,
                          max_combinations=6,
                          use_filter=True,
                          verbose=True,
                          early_stopping=False)

Average ComplEx Loss:   0.027317: 100%|█████| 10/10 [00:00<00:00, 13.46epoch/s]

Average ComplEx Loss:   1.970142: 100%|█████| 10/10 [00:00<00:00, 13.35epoch/s]

Average ComplEx Loss:   0.416925: 100%|█████| 10/10 [00:00<00:00, 11.64epoch/s]

Average ComplEx Loss:   0.164233: 100%|█████| 10/10 [00:01<00:00,  9.75epoch/s]

Average ComplEx Loss:   0.027751: 100%|█████| 10/10 [00:00<00:00, 11.64epoch/s]

Average ComplEx Loss:   0.401445: 100%|█████| 10/10 [00:00<00:00, 10.25epoch/s]

100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 28.94it/s]


In [18]:
best_model

In [19]:
save_model(best_model, './trained_models/ComplEx.pkl')
test_metrics["ComplEx"] = complEx_mrr_test
#del best_model

## View test metrics

In [20]:
test_metrics

{'RandomBaseline': {'mrr': 0.021060303181742464,
  'mr': 61.75,
  'hits_1': 0.0,
  'hits_3': 0.0,
  'hits_10': 0.0},
 'TransE': {'mrr': 0.25383823529411764,
  'mr': 66.25,
  'hits_1': 0.0,
  'hits_3': 0.5,
  'hits_10': 0.5},
 'DistMult': {'mrr': 0.016620549629846094,
  'mr': 60.25,
  'hits_1': 0.0,
  'hits_3': 0.0,
  'hits_10': 0.0},
 'ComplEx': {'mrr': 0.38231837606837604,
  'mr': 35.0,
  'hits_1': 0.25,
  'hits_3': 0.5,
  'hits_10': 0.5}}

In [21]:
# save test metrics to file
file = open("test_metrics.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(test_metrics,file)

# close file
file.close()